In [18]:
from tqdm.auto import tqdm
from call_model import call_chat_once
import asyncio
import httpx
from uuid import uuid4
import json
import datasets

from typing import Optional
import os
import csv
import nest_asyncio
nest_asyncio.apply()

In [19]:
from datasets import load_dataset

eval_dataset = load_dataset("csv", data_files="data/eval.csv")["train"]

In [20]:
print(eval_dataset)

Dataset({
    features: ['context', 'question', 'answer', 'source_doc', 'groundedness_score', 'groundedness_eval', 'relevance_score', 'relevance_eval', 'standalone_score', 'standalone_eval'],
    num_rows: 52
})


In [21]:
OUTPUT_FILE = "data/eval_rag_results_k=20_fetch=40_top=3.csv"

def load_existing_results(file_path):
    dataset = []
    existing_questions = set()
    if os.path.exists(file_path):
        with open(file_path, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing_questions.add(row["user_input"])
                # parse retrieved_contexts lại từ JSON
                row["retrieved_contexts"] = json.loads(row["retrieved_contexts"])
                dataset.append(row)
    return existing_questions, dataset


def append_result_to_csv(file_path, row, fieldnames):
    file_exists = os.path.exists(file_path)
    with open(file_path, "a", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    verbose: Optional[bool] = False
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    fieldnames = ["user_input", "retrieved_contexts", "response", "reference"]

    processed_questions, dataset = load_existing_results(OUTPUT_FILE)
    print(f"Đã có {len(processed_questions)} câu hỏi xử lý trước đó, sẽ bỏ qua chúng.")


    for i, example in enumerate(tqdm(eval_dataset)):
        question = example["question"]
        print(question)
        if question in processed_questions:
            continue
        
        payload = {
            "question": question,
            "session_id": str(uuid4()),
            "chat_history": []
        }

        # try:
        relevant_docs, answer = asyncio.run(call_chat_once(payload))
        # except Exception as e:
        #     print(f"Error from question: '{question}': {e}")

        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')

        row = {
            "user_input": question,
            "retrieved_contexts": json.dumps(
                [doc['kwargs'].get('page_content') for doc in relevant_docs],
                ensure_ascii=False
            ),
            "response": answer,
            "reference": example['answer']
        }
        append_result_to_csv(OUTPUT_FILE, row, fieldnames)
        dataset.append({
            "user_input": question,
            "retrieved_contexts": json.loads(row["retrieved_contexts"]),
            "response": answer,
            "reference": example['answer']
        })
    return dataset

In [22]:
data = run_rag_tests(eval_dataset=eval_dataset)


Đã có 0 câu hỏi xử lý trước đó, sẽ bỏ qua chúng.


  0%|          | 0/52 [00:00<?, ?it/s]

Who proposed the hybrid Vision Transformer model?


2025-08-13 16:23:44,540 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
  2%|▏         | 1/52 [00:13<11:14, 13.23s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'Who proposed the hybrid Vision Transformer model?', 'type': 'human', 'id': '40f94256-06a6-402b-adaf-046b5f17a068'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_CvccDBKC1xpAy19cAKKPFZgj', 'function': {'arguments': '{"query":"hybrid Vision Transformer model proposal"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--5aeac50c-783e-411e-b99c-264b2ad4ca2c-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'hybrid Vision Transformer model proposal'}, 'id': 'call_CvccDBKC1xpAy19cAKKPFZgj', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 356, 'output_tok

...

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What are the two types of guides for contributing to Gradio?', 'type': 'human', 'id': '3ec821a0-1a8a-419c-88ae-64f3a8091fb6'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_SpOGJTwAUc9w5MPoMDmVVOt0', 'function': {'arguments': '{"query":"types of guides for contributing to Gradio"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--8c78d6a7-025a-4900-9742-d773e7167250-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'types of guides for contributing to Gradio'}, 'id': 'call_SpOGJTwAUc9w5MPoMDmVVOt0', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 3

2025-08-13 16:34:45,653 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
 98%|█████████▊| 51/52 [11:14<00:13, 13.78s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What method is called to compute gradients in the training pipeline?', 'type': 'human', 'id': '7e9128a1-39b4-4d19-be03-30c47712c550'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_lOlAamGiIAySxGd2SG7hPwDH', 'function': {'arguments': '{"query":"compute gradients training pipeline"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--52f43dcd-4c56-4e77-8477-cc5270721381-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'compute gradients training pipeline'}, 'id': 'call_lOlAamGiIAySxGd2SG7hPwDH', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 360, 'o

2025-08-13 16:34:57,528 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
100%|██████████| 52/52 [11:26<00:00, 13.20s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'Who created the Falcon language models?', 'type': 'human', 'id': 'adb7875a-ba48-4da5-a263-5959e578bb3a'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_RVY3EkNc0x7Bq6s0pcnePtHJ', 'function': {'arguments': '{"query":"Falcon language models creator"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--de5940c0-062b-48f5-907e-36399524eb43-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'Falcon language models creator'}, 'id': 'call_RVY3EkNc0x7Bq6s0pcnePtHJ', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 355, 'output_tokens': 20, 'total_tokens': 375,

In [23]:
from ragas import EvaluationDataset
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    FactualCorrectness
)
evaluation_dataset = EvaluationDataset.from_list(data)

# Khởi tạo các metric
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]
# metrics = [FactualCorrectness()]

In [24]:
print(len(evaluation_dataset))

52


In [25]:
from app.config import configs

import os
os.environ['OPENAI_API_KEY'] = configs.OPENAI_API_KEY

result = evaluate(
    evaluation_dataset,
    metrics=metrics,
)

print(result) 

Evaluating:   0%|          | 0/208 [00:00<?, ?it/s]2025-08-13 16:34:59,747 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:34:59,926 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:34:59,974 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:35:00,026 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:35:00,047 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:35:00,085 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:35:00,098 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:35:00,674 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-13 16:35:01,101 - INFO - HTTP Request: POST

{'faithfulness': 0.6232, 'answer_relevancy': 0.7539, 'context_precision': 0.4679, 'context_recall': 0.4904}
